# Transfer Learning with TensorFlow Part 1: Feature Extraction

Transfer learning is leveraging a working model's existing architecture and learned patterns for our own problems.

There are two main benefits:

1. Can leverage an existing neural network architecture proven to work on problems similar to our own.
2. Can leverage a working neural network architecture which has already learned patterns on similar data to our own, then we can adapt those patterns to our own data.

## Downloading and becoming one with the data

In [1]:
# Get data (10% of 10 food classes from Food10)
import requests
import zipfile
import io
# Download the zip file using requests

url = "https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip"
response = requests.get(url)
# Unzip the file in memory and extract it

with zipfile.ZipFile(io.BytesIO(response.content)) as zip_ref:
    zip_ref.extractall("pizza_steak")  # Extract to a folder called 'pizza_steak'

In [2]:
# How many images in each folder?
import os

# Walk through 10 percent data directory and list number of files
for dirpath, dirnames, filenames in os.walk('../data/10_food_classes_10_percent/'):
    print(f"there are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'")

there are 2 directories and 0 images in '../data/10_food_classes_10_percent/'
there are 10 directories and 0 images in '../data/10_food_classes_10_percent/test'
there are 0 directories and 250 images in '../data/10_food_classes_10_percent/test\chicken_curry'
there are 0 directories and 250 images in '../data/10_food_classes_10_percent/test\chicken_wings'
there are 0 directories and 250 images in '../data/10_food_classes_10_percent/test\fried_rice'
there are 0 directories and 250 images in '../data/10_food_classes_10_percent/test\grilled_salmon'
there are 0 directories and 250 images in '../data/10_food_classes_10_percent/test\hamburger'
there are 0 directories and 250 images in '../data/10_food_classes_10_percent/test\ice_cream'
there are 0 directories and 250 images in '../data/10_food_classes_10_percent/test\pizza'
there are 0 directories and 250 images in '../data/10_food_classes_10_percent/test\ramen'
there are 0 directories and 250 images in '../data/10_food_classes_10_percent/tes

## Creating data loader (preparing the data)


In [3]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

IMAGE_SHAPE = (224, 224)
BATCH_SIZE = 32
train_dir = "../data/10_food_classes_10_percent/train/"
test_dir = "../data/10_food_classes_10_percent/test/"

train_data = image_dataset_from_directory(directory=train_dir,
                                          label_mode="categorical",
                                          batch_size=BATCH_SIZE,
                                          image_size=IMAGE_SHAPE,
                                          seed=42)
 
test_data = image_dataset_from_directory(directory=test_dir,
                                         label_mode="categorical",
                                         batch_size=BATCH_SIZE,
                                         image_size=IMAGE_SHAPE,
                                         seed=42)

Found 750 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


## Setting up callbacks (things to run while our model trains)

Callbacks are extra functionality you can add to your models to be performed during or after training. Some of the most popular callbacks:

* Tracking experiments with the TensorBoard callback
* Model checkpoin with the ModelCheckpoint callback
* Stopping a model from training (before it trains too long and overfits) with the EarlyStopping callback

In [5]:
# Create TensorBoard callback (functionized because we need to create a new one for each model)
import datetime

def create_tensorboard_callback(dir_name, experiment_name):
    log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
    print(f"Saving TensorBoard log files to: {log_dir}")

    return tensorboard_callback

## Creating models using TensorFlow Hub

In the past we've use TensorFlow to create our own models layers by layer from scratch.

Now we're going to do a similar process, except the majority of our model's layers are going to come fro TensorFlow Hub.

We can access pretrained models on: https://tfhub.dev/

Browsing the TensorFlow Hub page and sorting for image classification, we found the following feature vector model link:
https://www.kaggle.com/models/google/resnet-v2/tensorFlow2